# Homework 3: Thermal Wind and Ekman Transports #

In this problem set you will apply thermal wind balance and Ekman dynamics to real data.

First import numpy, matplotlib, xray, and gsw.

## 1. Thermal Wind from Hydrographic Data (35%) ##

In this problem, we explore the quantitative relation between lateral density gradients and geostrophic currents.

The thermal wind equations in component form are (using the Boussinesq approximation):

$$ \frac{\partial v_g}{\partial z} = \frac{1}{f} \frac{\partial b}{\partial x}
                                   = -\frac{g}{\rho_0 f} \frac{\partial \rho}{\partial x} \\
\frac{\partial u_g}{\partial z} = -\frac{1}{f} \frac{\partial b}{\partial y}
                                = \frac{g}{\rho_0 f} \frac{\partial \rho}{\partial y}
$$

where 

$$ b = - \frac{g({\rho - \rho_0})}{\rho_0}$$

### A) Conceptual question: does it matter whether we use $\rho$ (in situ density) or $\sigma$ (potential density) in the thermal wind equation? Why or why not?

### Load World Ocean Atlas Data ###

The following command will open the [2009 World Ocean Atlas](https://www.nodc.noaa.gov/OC5/WOA09/pr_woa09.html) via the web as an [xray dataset](http://xray.readthedocs.org/en/stable/data-structures.html#dataset).

This is the data we will use in the thermal wind equation.

In [4]:
woa = xray.open_dataset('http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NODC/.WOA09/'
                        '.Grid-1x1/.Annual/dods')

### B) Calculate the potential density field globally using gsw

First you have to convert practical salinity (``woa['salinity.s_an']``) to absolute salinity and temperature (``woa['temperature.t_an']``) to conservative temperature.

(Hint: to use gsw to operate on arrays, you might first want to apply the function ``xray.broadcast_arrays``. Read its documentation.)

(Another hint: to make this work with coordinate arrays (e.g. ``woa['lat']``), try multiplying them first by 1. Don't ask me why this is necessary...but it works.)

In [205]:
xray.broadcast_arrays?

### C) Plot sections of surface potential density in two different regions  ##

* Kuroshio Extension (logitude=170.5, latitude=30N - 50N)
* Antartic Circumpolar Current south of New Zealand (longitude=170.5, latitude=72S - 55S)


You may wish to turn $\sigma_0$ back in to an [xray DataArray](http://xray.readthedocs.org/en/stable/data-structures.html#creating-a-dataarray) to facilitate easier plotting.

Our goal is to use thermal wind to estimate the volume transport of ocean currents from observations of $T$ and $S$ alone.

### C) Math question: Integrate the thermal wind equation vertically (from depth -H to 0) and laterally (over horizontal distance L) to find an expression for the net geostrophic transport across a section.

You may simplify by assuming that $f$ (Coriolis parameter) is approximately constant over the section. Your final expression should contain an unknown constant of integration. Discuss what this constant represents. What does it imply about limitations of the thermal wind equation?

### D) Using your expression from above, estimate the net volume transport through the two sections.

Limit your calculatation to the _upper 2000 m_ and __assume the flow below 2000 m is zero__! (This assumption should allow you to eliminate the unknown constant of integration.)

If you can't figure out how to do this numerically (it's possible but requires care), then try to make some approximations that will allow you to arrive at an answer. If you explain your approximations carefully, you will still receive full credit.

# 2. Ekman Transport (35%)

Here we will estimate Ekman transport from real observational wind data.

## SCOW Wind Stress Data ##

Dataset:
http://cioss.coas.oregonstate.edu/scow/index.html

This dataset is not encoded properly, so we need more special code to process it. I have done this part for you.

In [109]:
# download SCOW data
!curl -O http://cioss.coas.oregonstate.edu/scow/data/monthly_fields/wind_stress_zonal_monthly_maps.nc
!curl -O http://cioss.coas.oregonstate.edu/scow/data/monthly_fields/wind_stress_meridional_monthly_maps.nc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36.9M  100 36.9M    0     0  3363k      0  0:00:11  0:00:11 --:--:-- 5370k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36.9M  100 36.9M    0     0  3418k      0  0:00:11  0:00:11 --:--:-- 3958k


In [133]:
# load data
scow_zonal = xray.open_dataset('wind_stress_zonal_monthly_maps.nc')
scow_merid = xray.open_dataset('wind_stress_meridional_monthly_maps.nc')
# each month is encoded as a different variable, annoying!
# have to manually average in time
dvars = scow_zonal.data_vars.keys()
taux = scow_zonal[dvars[0]]
tauy = scow_merid[dvars[0]]
for k in dvars[1:]:
    taux += scow_zonal[k]
    tauy = scow_merid[dvars[0]]
missing_value = -9999
taux = (taux / 12).where(taux>missing_value)
tauy = (tauy / 12).where(tauy>missing_value)

### A) Calculate and make maps of $U_{Ek}$ and $V_{Ek}$, the zonal and meridional Ekman volume transports (units m$^{2}$/s). How do the magnitudes of the two components compare?

You may make the Boussinesq approximation ($\rho \simeq \rho_0$).

Mask your data between $\pm$ 5$^\circ$ latitude, where $f$ becomes very small.

### B) Calculate and plot the zonally integrated meridional Ekman transport as a function of latitude.

Give your answer in Sv.

### C) Use your previous plot to estimate the net Ekman upwelling and downwelling in different latitude bands (also in Sv).

Choose the bands yourself based on on the locations of the minima and maxima of the curve.

## 3. Sverdrup Transport (30%)

In this problem we calculate the sverdrup flow in an idealized scenario.

We have a basin in Cartesian coordinates $(x,y)$ of dimensions $L_x$ x $L_y$. The lower left corner of the basin is at $(x,y) = (0,0)$ and the upper right corner is at $(x,y) = (L_x,\ L_y)$.

The zonal wind stress is given by

$$
\tau_x = -\tau_0 \cos\left ( \frac{2 \pi y}{L_y} \right )
$$

For simplicity, assume that $\beta = const = 10^{-11}$ m$^{-1}$ s$^{-1}$. 

### A) Plot $\tau_x(y)$

### B) Calculate and plot the zonally integrated meridional Sverdrup transport (units Sv) across the basin due to this wind stress?  

### C) Plot (contour) the barotropic transport streamfunction $\Psi(x,y)$ associated with this wind stress, _assuming $\Psi$ is zero on the Eastern boundary_.

### D) Discuss the implied western boundary currents. How does the boundary current transport depend on $\tau_0$? $L_y$?

## OPTIONAL EXTRA CREDIT (+20%) ##
## Repeat the Sverdrup streamfunction calculation for the Pacific using the SCOW wind stress data ##

Use real values of $\beta$.